In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)

In [3]:
import whoosh_utils

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [4]:
import gc
import warnings

import numpy as np
import polars as pl
import yaml
from tqdm import tqdm

warnings.filterwarnings("ignore")

2024-05-05 00:33:35.267890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 00:33:35.332139: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 00:33:35.809442: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

exp_0001


set seed: 46


In [5]:
train = pl.read_parquet("/kaggle/input/uspto-train-data/train_1000.parquet")
train.head(1)

publication_number,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,target_36,target_37,target_38,target_39,target_40,target_41,target_42,target_43,target_44,target_45,target_46,target_47,target_48,target_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-5292324-A""","""US-5662659-A""","""US-4959070-A""","""US-4813957-A""","""US-5752960-A""","""US-7186258-B2""","""US-4957505-A""","""US-4785810-A""","""US-5556400-A""","""US-5549614-A""","""US-5176701-A""","""US-4715373-A""","""US-7011666-B2""","""US-9277989-B2""","""US-6214015-B1""","""US-5234436-A""","""US-6206887-B1""","""US-2005143750-…","""US-7947049-B2""","""US-5217464-A""","""US-5207708-A""","""US-5312413-A""","""US-5163419-A""","""US-6811553-B2""","""US-5007913-A""","""US-8888845-B2""","""US-7422604-B2""","""US-5584304-A""","""US-2005149057-…","""US-6113606-A""","""US-4844065-A""","""US-4976716-A""","""US-5571113-A""","""US-5395378-A""","""US-5702400-A""","""US-5267553-A""","""US-2004193263-…","""US-4906247-A""","""US-6267768-B1""","""US-5876440-A""","""US-5066297-A""","""US-5728117-A""","""US-8574293-B2""","""US-4726367-A""","""US-5026393-A""","""US-2021244531-…","""US-6241737-B1""","""US-2009030424-…","""US-4655775-A""","""US-2022370195-…","""US-10744032-B2…"


In [6]:
import json

with open("/kaggle/input/cpc-counts/patent2cpc.json", "r") as f:
    patent2cpc = json.load(f)

In [7]:
with open("/kaggle/input/cpc-mapping/cpc2patents.json", "r") as f:
    cpc2patents = json.load(f)

In [18]:
for target_ids in train.iter_rows():
    target_ids = list(target_ids)[1:]
    assert len(target_ids) == 50

    inner_set = set()
    outer_set = set()

    all_cpcs = set(sum([patent2cpc[target_id] for target_id in target_ids], []))
    accepted_cpcs = []
    while True:
        best_cpc = None
        best_ratio = 1e10
        best_inners = []
        best_outers = []
        remove_cpcs = []
        for cpc in all_cpcs:
            inners = []
            outers = []
            for patent in cpc2patents[cpc]:
                if patent in target_ids:
                    if patent not in inner_set:
                        inners.append(patent)
                else:
                    if patent not in outer_set:
                        outers.append(patent)
            if len(inners) <= 1:
                remove_cpcs.append(cpc)
                continue

            ratio = len(outers) / len(inners)
            if ratio < best_ratio:
                best_ratio = ratio
                best_cpc = cpc
                best_inners = inners
                best_outers = outers

        for cpc in remove_cpcs:
            all_cpcs.remove(cpc)
        if best_cpc is None:
            break

        for patent in best_inners:
            inner_set.add(patent)
        for patent in best_outers:
            outer_set.add(patent)
        accepted_cpcs.append(best_cpc)

        if len(accepted_cpcs) == 15:
            break

    print(f"accept_cpcs: {len(accepted_cpcs)}, inner: {len(inner_set)}, outer: {len(outer_set)}")

accept_cpcs: 6, inner: 47, outer: 2334
accept_cpcs: 7, inner: 17, outer: 3558
accept_cpcs: 12, inner: 40, outer: 3814
accept_cpcs: 13, inner: 43, outer: 6182
accept_cpcs: 11, inner: 34, outer: 3127
accept_cpcs: 10, inner: 38, outer: 4738
accept_cpcs: 15, inner: 35, outer: 48621
accept_cpcs: 11, inner: 47, outer: 9817
accept_cpcs: 14, inner: 46, outer: 31567
accept_cpcs: 5, inner: 13, outer: 3713
accept_cpcs: 15, inner: 41, outer: 13081
accept_cpcs: 15, inner: 47, outer: 4513
accept_cpcs: 15, inner: 41, outer: 57926
accept_cpcs: 6, inner: 42, outer: 1894
accept_cpcs: 15, inner: 34, outer: 5587
accept_cpcs: 14, inner: 42, outer: 14783
accept_cpcs: 12, inner: 31, outer: 14863
accept_cpcs: 15, inner: 44, outer: 1978
accept_cpcs: 9, inner: 33, outer: 1401
accept_cpcs: 8, inner: 33, outer: 3106
accept_cpcs: 13, inner: 38, outer: 35189
accept_cpcs: 7, inner: 46, outer: 3525
accept_cpcs: 13, inner: 43, outer: 57592
accept_cpcs: 6, inner: 41, outer: 2679
accept_cpcs: 11, inner: 48, outer: 5596


KeyboardInterrupt: 

15
38 2155
